In [39]:
import ifcopenshell
import ifcopenshell.geom as geom

import ifcopenshell.api.alignment

file = ifcopenshell.open("F:/IFC-Rail-Unit-Test-Reference-Code/alignment_testset/IFC-WithGeneratedGeometry/GENERATED__HorizontalAlignment_HelmertCurve_100.0_1000_300_1_Meter.ifc")

inst = file.by_type('IfcCompositeCurve')[0]
print(inst)

# Create a IfcBuildingElementProxy
# Set the ObjectPlacement using an IfcLinearPlacement where the IfcPointByDistanceExpression references the composite curve
# Create a new IfcCompositeCurve with an IfcCurveSegment and IfcThirdOrderPolynomialSpiral that is NOT a Bloss curve (use a fancy spiral)
# Create an IfcSweptDiskSolid using the crazy curve
# Save the model into a new file

A0 = -2.5 #-0.25 # -4
A1 = 0
A2 = 10.0 #1 #0.33

parent_curve = file.createIfcSecondOrderPolynomialSpiral(
    Position=file.createIfcAxis2Placement2D(
        Location=file.createIfcCartesianPoint((0.0, 0.0)), RefDirection=file.createIfcDirection((1.0, 0.0))
    ),
    QuadraticTerm=A2,
    LinearTerm=A1 if A1 != 0.0 else None,
    ConstantTerm=A0 if A0 != 0.0 else None,
)

curve_segment = file.create_entity(
    type="IfcCurveSegment",
    Transition="DISCONTINUOUS",
    Placement=file.createIfcAxis2Placement2D(
        Location=file.createIfcCartesianPoint((0.0,0.0)),
        RefDirection=file.createIfcDirection((1.0,0.0)),
    ),
    SegmentStart=file.createIfcLengthMeasure(-200.0),
    SegmentLength=file.createIfcLengthMeasure(400.0),
    ParentCurve=parent_curve,
)

composite_curve = file.createIfcCompositeCurve(SelfIntersect=False,Segments=[curve_segment,])

line_parent_curve = file.createIfcLine(Pnt=file.createIfcCartesianPoint((0.0,0.0)),
                                       Dir=file.createIfcVector(file.createIfcDirection(DirectionRatios=((1.0,0.0))),Magnitude=1.0))
curve_segment = file.create_entity(
    type="IfcCurveSegment",
    Transition="DISCONTINUOUS",
    Placement=file.createIfcAxis2Placement2D(
        Location=file.createIfcCartesianPoint((0.0,0.0)),
        RefDirection=file.createIfcDirection((1.0,0.0)),
    ),
    SegmentStart=file.createIfcLengthMeasure(-200.),
    SegmentLength=file.createIfcLengthMeasure(400.),
    ParentCurve=line_parent_curve,
)

gradient_curve = file.createIfcGradientCurve(SelfIntersect=False,Segments=[curve_segment,],BaseCurve=composite_curve,EndPoint=None)

object_placement = file.createIfcLinearPlacement(
                    RelativePlacement=file.createIfcAxis2PlacementLinear(
                       Location=file.createIfcPointByDistanceExpression(
                       DistanceAlong=file.createIfcLengthMeasure(50.0),
                       OffsetLateral=None,
                       OffsetVertical=None,
                       OffsetLongitudinal=None,
                       BasisCurve=inst,
                   )
               ),
            CartesianPosition=None,
        )

swept_disk_solid = file.createIfcSweptDiskSolid(Directrix=gradient_curve,Radius=0.05)

context = ifcopenshell.util.representation.get_context(file,"Model","Body","MODEL_VIEW")
if context == None:
    geometric_representation_context = ifcopenshell.api.context.add_context(file, context_type="Model")
    context = ifcopenshell.api.context.add_context(
        file,
        context_type="Model",
        context_identifier="Body",
        target_view="MODEL_VIEW",
        parent=geometric_representation_context,
    )    
    
shape_representation = file.createIfcShapeRepresentation(ContextOfItems=context,RepresentationIdentifier="Body",RepresentationType="AdvancedSweptSolid",Items=(swept_disk_solid,))

proxy = file.createIfcBuildingElementProxy(
    GlobalId=ifcopenshell.guid.new(),
    ObjectPlacement=object_placement,
    Name="Roadside Art"
)

ifcopenshell.api.geometry.assign_representation(file,proxy,shape_representation)

project = file.by_type("IfcProject")[0]
alignment = file.by_type("IfcAlignment")[0]
ifcopenshell.api.aggregate.assign_object(file,products=[alignment],relating_object=project)

site = file.by_type("IfcRailway")[0]
ifcopenshell.api.spatial.assign_container(file,products=[proxy],relating_structure=site)

file.write("C:\\Users\\bricer\\OneDrive - Washington State Department of Transportation\\Desktop\\GenericSpiral.ifc")
print("Done")

#35=IfcCompositeCurve((#36,#48,#56),.F.)
Done
